In [1]:
# -*- coding: utf-8 -*-
"""
@author: Alain Plantec

Voici un skelette possible.
Vous devez programmez les classes contenues dans ce squelette.
Les fonctions et leurs paramètres ainsi que les variables contenues dans le classes
ont du sens par rapport à une version programmée pour la préparation du projet.
Votre version sera forcément différente.
Donc, vous pouvez ajouter/retirer des variables et/ou des fonctions et/ou des paramètres.

"""
try:  # import as appropriate for 2.x vs. 3.x
    import tkinter as tk
    import tkinter.messagebox as tkMessageBox
except:
    import Tkinter as tk
    import tkMessageBox

from sokobanXSBLevels import *
from enum import Enum
class Victory(object): #But de la classe est de savoir quand on gagne
    def __init__(self, level, initial_box_count):
        self.level = level
        self.initial_box_count = initial_box_count #le nombre de box initiale

    def check_victory(self):
        current_box_count = sum(1 for row in self.level.movableMatrix for box in row if isinstance(box, Box) and box.onGoal) # fais la somme des intance de box se trouvrant dans la matric
        if current_box_count == self.initial_box_count:                                                                      # Movable et voit si onGOal est true il fais la somme en ajoutan 1
            self.level.display_victory_message()

    


class Direction(Enum):
    Up = 1
    Down = 2
    Left = 3
    Right = 4
class Position(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return 'Position(' + str(self.x) + ',' + str(self.y) + str(')')

    def getX(self):
        return self.x

    def getY(self):
        return self.y

    def positionTowards(self, direction, offset): # Prend deux parametre le premier une direction de la class Direction, offset est le pas 
        if direction == Direction.Up:
            return Position(self.x, self.y - offset) # self.y-offset car l'origine du repere est en haut et vers la gauche donc monte
        elif direction == Direction.Down:
            return Position(self.x, self.y + offset)
        elif direction == Direction.Left:
            return Position(self.x - offset, self.y)
        elif direction == Direction.Right:
            return Position(self.x + offset, self.y)

    def isValidInWharehouse(self, warehouse): 
        return warehouse.isPositionValid(self)
    def isTarget(self,warehouse):
        return warehouse.BoxOnTarget(self)


class Floor(object): #Pas eu le temps
    def __init__(self):
        pass

    def isMovable(self):
        return False

    def canBeCovered(self):
        return True

    def xsbChar(self):
        return ' '

    def isFreePlace(self):
        return True


class Goal(object): 
    def __init__(self, canvas, position):
        self.position = position
        self.height = 64  
        self.width = 64  
        self.canvas = canvas
        self.image = None

    def getHeight(self):
        return self.height

    def getWidth(self):
        return self.width

    def canBeCovered(self):
        return True

    def xsbChar(self):
        return '.'

    def isFreePlace(self):
        return False
    def create_image(self,x,y,is_covered=False): #Cette methode permet d'ajouter l'image(boxontarget) puis la cree False car au debut pas couvert 
        if not is_covered:
            self.image=tk.PhotoImage(file='goal.png')
            self.image_id = self.canvas.create_image(x * 64, y * 64, image=self.image, anchor=tk.NW, tags="static")
        else:
            self.image=tk.PhotoImage(file='boxOnTarget.png')
            self.image_id = self.canvas.create_image(x * 64, y * 64, image=self.covered_image, anchor=tk.NW, tags="movable")

        return self.image_id
    


class Wall(object):
    def __init__(self, canvas, position):
        self.position = position
        self.height = 64  
        self.width = 64  
        self.canvas = canvas
        self.image = tk.PhotoImage(file='wall.png')

    def getHeight(self):
        return self.height

    def getWidth(self):
        return self.width

    def isMovable(self):
        return False

    def canBeCovered(self):
        return False

    def xsbChar(self):
        return '#'

    def isFreePlace(self):
        return False
    def create_image(self,x,y): 
        self.canvas.create_image(x * 64, y * 64, image=self.image, anchor=tk.NW, tags="static")


class Box(object):
    def __init__(self, canvas, warehouse, position, onGoal):
        self.position = position
        self.height = 64  
        self.width = 64  
        self.canvas = canvas
        self.onGoal = onGoal
        self.image = None
        self.image_id=None

    def getHeight(self):
        return self.height

    def getWidth(self):
        return self.width
    def set_box_target(self,on_goal): #De base Une box a un Ongoal = False si les position du box == position du Goal Dans le level nous metton self.box.set_box_target(True) 
        if on_goal:                     #else False et change l'image en fonction
            self.onGoal = True
            self.image = tk.PhotoImage(file='boxOnTarget.png')
        else:
            self.onGoal = False
            self.image = tk.PhotoImage(file='box.png')

    def isMovable(self):
        return True

    def canBeCovered(self):
        return False

    def moveTowards(self, direction): #Calcule la nouvelle position de la box
        new_position = self.position.positionTowards(direction, 1)
        self.position = new_position

    def xsbChar(self):
        if self.onGoal:
            return '*'
        else:
            return '$'

    def isFreePlace(self):
        return False
    

    def create_image(self,x,y): 
        self.image_id=self.canvas.create_image(x * 64, y * 64, image=self.image, anchor=tk.NW, tags="movable")
        return self.image_id
class Mover(object):
    def __init__(self, canvas, warehouse, position, onGoal):
        self.position = position
        self.height = 64  
        self.width = 64  
        self.canvas = canvas
        self.onGoal = onGoal
        self.image = tk.PhotoImage(file='playerDown.png')
        self.warehouse=warehouse
        self.image_id = None
         

    def getHeight(self):
        return self.height

    def getWidth(self):
        return self.width

    def isMoveable(self):
        return True

    def moveInCanvas(self, direction):
        new_position = self.position.positionTowards(direction, 1)
        self.position = new_position

    def canMove(self, direction):
        new_position = self.position.positionTowards(direction, 1)
        return new_position.isValidInWharehouse(self.warehouse)

    def moveTowards(self, direction):
        new_position = self.position.positionTowards(direction, 1)
        self.position = new_position


    def push(self, direction): #Parametre mais pas reussi a implemente
        self.setupImageForDirection(direction)
        if not self.canMove(direction):
            self.startImpossiblePushAnimation()
            return
        self.moveTowards(direction)

    def xsbChar(self):
        if self.onGoal:
            return '+'
        else:
            return '@'

    def isFreePlace(self):
        return False

    def mover_deplacement(self,direction): #Gere les changements d'image en Fonction du mouvement du player 
        if(direction==Direction.Up):
            self.image=tk.PhotoImage(file='playerUp.png')   
        elif(direction==Direction.Down):
            self.image=tk.PhotoImage(file='playerDown.png')
        elif(direction==Direction.Left):
            self.image=tk.PhotoImage(file='playerLeft.png')
        else:
            self.image=tk.PhotoImage(file='playerRight.png')

    def impossiblePushAnimation(self):
        # Not implemented
        pass
    def create_image(self, x, y):
        self.image_id = self.canvas.create_image(x * 64, y * 64, image=self.image, anchor=tk.NW, tags="movable")
        return self.image_id


class WarehousePlan(object):
    def __init__(self):
        self.rawMatrix = []

    def appendRow(self, row):
        for line in row:
            s = []
            for e in line:
                if e is None:
                    s.append(' ')
                else:
                    s.append(e) 
            self.rawMatrix.append(s)

    def asXsbMatrix(self):
        return xsbMatrix(self.rawMatrix)
    def isPositionValid(self, position):
        x, y = position.getX(), position.getY()
        return 0 <= y < len(self.rawMatrix) and 0 <= x < len(self.rawMatrix[y]) and self.rawMatrix[y][x] != '#' # Il permet juste de detecter les murs et interdire de le depasser car le boolean != est 
    def BoxOnTarget(self,position):                                                                             #True quand nous somme pas sur le mur
        x, y = position.getX(), position.getY()
        return 0 <= y < len(self.rawMatrix) and 0 <= x < len(self.rawMatrix[y]) and self.rawMatrix[y][x] == '.'# pareil mais pour les goals True quand on est sur le goal

class Level(object):
    def __init__(self, root, xsbMatrix):
        self.playerX = 0  
        self.playerY = 0  
        self.root = root
        self.staticMatrix = [] #ELEMENT STATiC QUI BOUGE PAS MUR ET GOAL
        self.movableMatrix = [] #ELEMENT BOUGEABLE MOVER ET BOX
        self.warehouse = WarehousePlan()
        self.warehouse.appendRow(xsbMatrix)
        self.height, self.width = self.calculate_dimensions(self.warehouse.rawMatrix)
        self.canvas = tk.Canvas(self.root, width=self.width, height=self.height, bg="gray")
        self.canvas.pack()
        nbrows = len(self.warehouse.rawMatrix)
        nbcolumns = max(len(line) for line in self.warehouse.rawMatrix)
        for lineIdx in range(nbrows):
            self.staticMatrix.append([])
            self.movableMatrix.append([])
            for elemIdx in range(nbcolumns):
                self.staticMatrix[lineIdx].append(None)
                self.movableMatrix[lineIdx].append(None)
        self.initWarehouseFromXsb(self.warehouse.rawMatrix)
        self.root.bind("<Key>", self.keypressed)

    def calculate_dimensions(self, xsbMatrix):
        nbrows = len(xsbMatrix)
        nbcolumns = max(len(line) for line in xsbMatrix)
        return nbrows * 64, nbcolumns * 64

    def initWarehouseFromXsb(self, xsbMatrix):
        initial_box_count = 0
        for lineIdx in range(len(xsbMatrix)):
            for elemIdx in range(len(xsbMatrix[lineIdx])):
                position = Position(elemIdx, lineIdx)
                element_char = xsbMatrix[lineIdx][elemIdx]
                if element_char == '#':
                    wall = Wall(self.canvas, position)
                    self.staticMatrix[lineIdx][elemIdx] = wall
                    wall.create_image(elemIdx,lineIdx)
                    self.wall=wall
                elif element_char == '$':
                    box = Box(self.canvas, self.warehouse, position, False)
                    self.movableMatrix[lineIdx][elemIdx] = box
                    box.set_box_target(False)
                    box.create_image(elemIdx,lineIdx)
                    initial_box_count+=1
                elif element_char == '.':
                    goal = Goal(self.canvas, position)
                    self.staticMatrix[lineIdx][elemIdx] = goal
                    goal.create_image(elemIdx,lineIdx)
                elif element_char == '@':
                    mover = Mover(self.canvas, self.warehouse, position, False)
                    self.movableMatrix[lineIdx][elemIdx] = mover
                    mover.create_image(elemIdx,lineIdx)
                    self.playerX=elemIdx
                    self.playerY=lineIdx

        
        self.victory = Victory(self, initial_box_count)
                    
        self.canvas.tag_raise("movable", "static")
    def display_victory_message(self):
        tkMessageBox.showinfo("Victory", "Bravo!")
        self.canvas.quit()


    def keypressed(self, event): # PRINCIPE EST DE CALCULER LES POSITION AVANT ET APRES MODIFICATION DE LA BOX ET DU MOVER
        self.gap = 1
        mover = self.movableMatrix[self.playerY][self.playerX]
        box=None
        old_mover_position=mover.position
        if event.keysym == 'Up': 
            mover.moveInCanvas(Direction.Up)
            mover.mover_deplacement(Direction.Up)
            if self.movableMatrix[mover.position.y][mover.position.x] is not None:
                old_box_position = mover.position
                box = self.movableMatrix[mover.position.y][mover.position.x]     
                box.moveTowards(Direction.Up)
        elif event.keysym == 'Down':
            mover.moveInCanvas(Direction.Down)
            mover.mover_deplacement(Direction.Down)
            if self.movableMatrix[mover.position.y][mover.position.x] is not None:
                old_box_position = mover.position
                box = self.movableMatrix[mover.position.y][mover.position.x] 
                box.moveTowards(Direction.Down)
        elif event.keysym == 'Left':
            mover.moveInCanvas(Direction.Left)
            mover.mover_deplacement(Direction.Left)
            if self.movableMatrix[mover.position.y][mover.position.x] is not None:
                old_box_position = mover.position
                box = self.movableMatrix[mover.position.y][mover.position.x] 
                box.moveTowards(Direction.Left)
        elif event.keysym == 'Right':
            mover.moveInCanvas(Direction.Right)
            mover.mover_deplacement(Direction.Right)
            if self.movableMatrix[mover.position.y][mover.position.x] is not None:
                old_box_position = mover.position
                box = self.movableMatrix[mover.position.y][mover.position.x]
                box.moveTowards(Direction.Right)

        new_mover_position=mover.position
        if box!=None: new_box_position=box.position # DETERMINE LA NOUVELE POSITION SI BOX EST DIFFERENT DE NONE
        if mover.position.isValidInWharehouse(self.warehouse) and box==None: #PARTIE 1 ON SE DEPLACE SANS PROBLEME ET OBSTACLE
            self.movableMatrix[new_mover_position.y][new_mover_position.x] = mover
            self.movableMatrix[old_mover_position.y][old_mover_position.x] = None
            self.canvas.delete(mover.image_id)
            mover.create_image(new_mover_position.x,new_mover_position.y)
            self.playerY=new_mover_position.y
            self.playerX=new_mover_position.x
        elif mover.position.isValidInWharehouse(self.warehouse) and (box.position.isValidInWharehouse(self.warehouse) and self.movableMatrix[new_box_position.y][new_box_position.x]==None):#PARTIE 2 SI LA NOUVELLE DU BOX EST VALID ET QUE NOUS AVONS PAS DE NOUVELLE BOX DEVANT
            if box.position.isTarget(self.warehouse):box.set_box_target(True)
            else: box.set_box_target(False)
            self.movableMatrix[new_box_position.y][new_box_position.x] = box
            self.movableMatrix[old_box_position.y][old_box_position.x] = None
            self.canvas.delete(box.image_id)
            box.create_image(new_box_position.x,new_box_position.y)
            self.movableMatrix[new_mover_position.y][new_mover_position.x] = mover
            self.movableMatrix[old_mover_position.y][old_mover_position.x] = None
            self.canvas.delete(mover.image_id)
            mover.create_image(new_mover_position.x,new_mover_position.y)
            self.playerY=new_mover_position.y
            self.playerX=new_mover_position.x
        elif mover.position.isValidInWharehouse(self.warehouse) and (box.position.isValidInWharehouse(self.warehouse) and self.movableMatrix[new_box_position.y][new_box_position.x]!=None): #PART 3 NOUS AVONS UNE BOX DEVANT
            box.position.x=old_box_position.x
            box.position.y=old_box_position.y
            mover.position.y = old_mover_position.y
            mover.position.x = old_mover_position.x
            mover.create_image(mover.position.x, mover.position.y)
            print("Box devant")
        elif mover.position.isValidInWharehouse(self.warehouse) and (box.position.isValidInWharehouse(self.warehouse)==False and self.movableMatrix[new_box_position.y][new_box_position.x]==None): #LE CAS OU NOTRE BOX EST DEVANT UN MUR ET QUE NOUS POUVONS NI BOUGER LA BOX NI BOUGER LE PLAYER
            print("Box bloc")
            box.position.x=old_box_position.x
            box.position.y=old_box_position.y
            #box.create_image(box.position.x,box.position.y)
            mover.position.y = old_mover_position.y
            mover.position.x = old_mover_position.x
            mover.create_image(mover.position.x, mover.position.y)
            
        else:
            mover.position.y = old_mover_position.y
            mover.position.x = old_mover_position.x
            mover.create_image(mover.position.x, mover.position.y)
            print("position invalid")
        self.victory.check_victory()


                      
class Sokoban(object):
    def __init__(self):
        self.root = tk.Tk()
        self.root.resizable(False, False)
        self.root.title("Sokoban")
        print('Sokoban: ' + str(len(SokobanXSBLevels)) + ' levels')
        self.level = Level(self.root, SokobanXSBLevels[1])

    def play(self):
        self.root.mainloop()

Sokoban().play()

Sokoban: 101 levels
